In [1]:
import warnings
warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", FutureWarning)
warnings.simplefilter("ignore", RuntimeWarning)

import sys
sys.path.append('../../dstorch/')

import pandas as pd

%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
from dstorch import io, prepare_data, dataset, transforms, losses, models, predict
from dstorch.train import train
from dstorch.utils import variable
import torch.backends.cudnn as cudnn
import torch 
import torch.nn as nn
from torch.optim import Adam

In [3]:
TRAIN_PATH = '../../kaggle-dsbowl-2018-dataset-fixes/stage1_train/'
TEST_PATH = '../../stage1_test/'

ids, images, masks = io.read_train_data(TRAIN_PATH, d=3)
test_ids, test_images = io.read_test_data(TEST_PATH, d=3)

HBox(children=(IntProgress(value=0, description='Reading train data..', max=664), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Reading test data..', max=65), HTML(value='')))

In [4]:
classes = pd.read_csv("../../classes.csv")

images = prepare_data.invert_images(classes, images, ids)

In [5]:
[train_images, train_masks, train_ids], [val_images, val_masks, val_ids] = prepare_data.train_val_split(
    classes, ids, images, masks, ids)

In [6]:
train_transform = transforms.DualCompose(
    [
        transforms.RandomCrop((256, 256)),
        transforms.VerticalFlip(1),
        transforms.HorizontalFlip(1),
        transforms.Transpose(1),
        transforms.ShiftScaleRotate(1),
        transforms.ImageOnly(transforms.RandomHueSaturationValue()),
        transforms.ImageOnly(transforms.RandomBrightness()),
        transforms.ImageOnly(transforms.RandomContrast()),
        transforms.ImageOnly(transforms.Normalize())
    ]
)

val_transform = transforms.DualCompose(
    [
        transforms.RandomCrop((256, 256)),
        transforms.ImageOnly(transforms.Normalize())
    ]
)

train_loader = dataset.make_loader(train_images, train_masks, train_ids, 16, transform=train_transform)
val_loader = dataset.make_loader(val_images, val_masks, val_ids, 1, transform=val_transform)

In [8]:
model = models.TernausNet34(num_classes=1, num_filters=16)
loss = losses.BCEDiceLoss()

if torch.cuda.is_available():
    model = nn.DataParallel(model).cuda()    
    cudnn.benchmark = True

RuntimeError: cuda runtime error (46) : all CUDA-capable devices are busy or unavailable at /pytorch/torch/lib/THC/generic/THCStorage.cu:58

In [ ]:
train(
    init_optimizer=lambda lr: Adam(model.parameters(), lr=lr),
    model=model,
    criterion=loss,
    train_loader=train_loader,
    val_loader=val_loader,
    n_epochs=50, 
    batch_size=32,
    lr=0.001
)

HBox(children=(IntProgress(value=0, max=1216), HTML(value='')))

In [104]:
loader = dataset.make_loader(val_images, masks=val_masks, ids=val_ids,
                         batch_size=1, transform=val_transform, shuffle=False, mode='predict')

In [105]:
for batch in loader:
    pass

In [106]:
model(variable(batch[0], volatile=True))

Variable containing:
( 0 , 0 ,.,.) = 
  -5.2279  -8.3650  -9.5367  ...    4.5529   3.8050   2.3795
  -7.7909 -11.4215 -12.4779  ...    6.8408   5.6576   3.5199
  -8.1152 -12.0261 -13.5390  ...    7.9001   6.4334   3.9178
            ...               ⋱              ...            
  -7.1068 -10.9750 -12.3905  ...  -14.4827 -14.2007  -9.9386
  -6.6166 -10.3494 -11.7766  ...  -13.5187 -12.6015  -8.6354
  -5.0839  -7.7350  -9.0471  ...  -10.0162  -8.9267  -5.8874
[torch.cuda.FloatTensor of size 1x1x256x128 (GPU 0)]

In [49]:
test_transform = transforms.DualCompose(
    [
        transforms.ImageOnly(transforms.Normalize())
    ]
)

test_predictions, test_names = predict.predict(model, test_images, test_ids, test_transform, 1)

Predict:   0%|          | 0/65 [00:00<?, ?it/s]


RuntimeError: invalid argument 0: Sizes of tensors must match except in dimension 1. Got 86 and 84 in dimension 3 at /pytorch/torch/lib/THC/generic/THCTensorMath.cu:111